# Liver Model

## Import some things

In [1]:
from Liver_Generator import Data_Generator, os, plot_scroll_Image, dice_coef_3D

Using TensorFlow backend.
c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qi

In [2]:
%matplotlib notebook

In [3]:
data_path = os.path.join('..','Data','Numpy_Arrays')
train_path = os.path.join(data_path,'Train')
validation_path = os.path.join(data_path,'Validation')

In [4]:
args = {'batch_size':5,'mean_val':81,'std_val':31,'on_vgg':True}
train_generator = Data_Generator(train_path, shuffle=True, **args) # mean_val=81,std_val=30
validation_generator = Data_Generator(validation_path, shuffle=True, **args) # mean_val=81,std_val=30

## Normalize images about the mask to be mean 0, std 1

In [5]:
#train_generator.get_mean_std_val() # This will calculate the mean and std for you and set it in the model
print(train_generator.mean_val)

81


In [6]:
x,y = train_generator.__getitem__(0)

In [ ]:
plot_scroll_Image(x[...,0])

### Alright, lets make our model!

In [7]:
from Easy_VGG16_UNet.Keras_Fine_Tune_VGG_16_Liver import VGG_16
from Visualizing_Model.Visualing_Model import visualization_model_class
from keras.optimizers import Adam

In [8]:
network = {'Layer_0': {'Encoding': [64, 64], 'Decoding': [32]},
           'Layer_1': {'Encoding': [128, 128], 'Decoding': [64]},
           'Layer_2': {'Encoding': [256, 256, 256], 'Decoding': [256]},
           'Layer_3': {'Encoding': [512, 512, 512], 'Decoding': [256]},
           'Layer_4': {'Encoding': [512, 512, 512]}}
VGG_model = VGG_16(network=network, activation='relu',filter_size=(3,3))
VGG_model.make_model()
VGG_model.load_weights()
new_model = VGG_model.created_model

W1018 11:07:15.629978 18116 deprecation_wrapper.py:119] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:548: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1018 11:07:15.651982 18116 deprecation_wrapper.py:119] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4439: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1018 11:07:15.680002 18116 deprecation_wrapper.py:119] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4274: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1018 11:07:15.811981 18116 deprecation_wrapper.py:119] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:2246: The name tf.image.resize_nearest_neigh

Layer_3
Layer_2
Layer_1
Layer_0


W1018 11:07:16.187028 18116 deprecation_wrapper.py:119] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:191: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1018 11:07:16.189029 18116 deprecation_wrapper.py:119] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:198: The name tf.ConfigProto is deprecated. Please use tf.compat.v1.ConfigProto instead.

W1018 11:07:16.189989 18116 deprecation_wrapper.py:119] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:204: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [ ]:
Visualizing_Class = visualization_model_class(model=new_model, save_images=True, verbose=True)

In [ ]:
Visualizing_Class.define_desired_layers(['block1_conv1','Output'])

In [ ]:
Visualizing_Class.predict_on_tensor(x)

In [ ]:
Visualizing_Class.plot_activations()

In [9]:
new_model.compile(Adam(lr=1e-3),loss='categorical_crossentropy', metrics=['accuracy',dice_coef_3D])

W1018 11:07:22.318512 18116 deprecation_wrapper.py:119] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\keras\optimizers.py:793: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



In [10]:
new_model.fit_generator(train_generator,epochs=5, workers=20, max_queue_size=100, validation_data=validation_generator,
                       steps_per_epoch=100)

W1018 11:07:23.592926 18116 deprecation.py:323] From c:\users\bmanderson\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/5
100/100 [==============================] - 90s 904ms/step - loss: 0.9393 - acc: 0.9416 - dice_coef_3D: 0.0020 - val_loss: 1.3717 - val_acc: 0.9149 - val_dice_coef_3D: 1.0161e-09
Epoch 2/5
100/100 [==============================] - 57s 571ms/step - loss: 0.8712 - acc: 0.9459 - dice_coef_3D: 1.6535e-09 - val_loss: 1.3717 - val_acc: 0.9149 - val_dice_coef_3D: 1.0161e-09
Epoch 3/5
100/100 [==============================] - 57s 571ms/step - loss: 0.8775 - acc: 0.9456 - dice_coef_3D: 1.5806e-09 - val_loss: 1.3717 - val_acc: 0.9149 - val_dice_coef_3D: 1.0161e-09
Epoch 4/5
100/100 [==============================] - 70s 697ms/step - loss: 0.8885 - acc: 0.9449 - dice_coef_3D: 1.5963e-09 - val_loss: 1.3717 - val_acc: 0.9149 - val_dice_coef_3D: 1.0161e-09
Epoch 5/5
100/100 [==============================] - 56s 556ms/step - loss: 0.8702 - acc: 0.9460 - dice_coef_3D: 1.5809e-09 - val_loss: 1.3717 - val_acc: 0.9149 - val_dice_coef_3D: 1.0161e-09
